In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from triangle import triangulate
from utils import is_extreme_boundary
import plotly

### Load precomputed integrals

In [ ]:
lagrange_linear_poisson_matrix = dill.load(
    open("../calculations/lagrange_linear_poisson_matrix", "rb")
)
lagrange_linear_poisson_right_values = dill.load(
    open("../calculations/lagrange_linear_poisson_right_values", "rb")
)

lagrange_linear_poisson_matrix_curvilinear = dill.load(
    open("../calculations/lagrange_linear_weak_form_functional_cubic_hermite", "rb")
)
lagrange_linear_poisson_right_values_curvilinear = dill.load(
    open("../calculations/lagrange_linear_weak_form_right_part_cubic_hermite", "rb")
)

mapping_function = dill.load(
    open("../calculations/lagrange_linear_mapping_function", "rb")
)

interpolation_function = dill.load(open("../calculations/lagrange_linear_basis", "rb"))


### Initialize the domain

### Parametri curve

In [ ]:
t = sympy.symbols("t")
x = 1*sympy.cos(t)
y = 1*sympy.sin(t)

In [ ]:
t_param = np.linspace(0, 2 * np.pi, 4, endpoint=False)

points = np.stack((sympy.lambdify(t, x)(t_param), sympy.lambdify(t, y)(t_param))).T

seg = np.array(
    [[j, j + 1] for j in range(points.shape[0] - 1)] + [[points.shape[0] - 1, 0]]
)

In [ ]:
directons = np.stack([sympy.lambdify(t, x.diff(t))(t_param), sympy.lambdify(t, y.diff(t))(t_param)]).T

In [ ]:
t_param, dt = np.linspace(0, 2 * np.pi, 1000, endpoint=False, retstep=True)
dense_x, dense_y = np.stack([sympy.lambdify(t, x)(t_param), sympy.lambdify(t, y)(t_param)])
length = np.sqrt(np.diff(dense_x/dt)**2 + np.diff(dense_y/dt)**2).sum()*dt
directons /= length/10

### Run triangulation

In [ ]:
cndt = triangulate({"vertices": points, "segments": seg}, opts="pq30a0.3594neo1s")
points = cndt["vertices"]
triangles = cndt["triangles"]
neighbours = cndt["neighbors"]
vertex_marker_is_boundary = cndt["vertex_markers"]
edges = cndt["edges"]
edge_marker_is_boundary = cndt["edge_markers"]

print(f"Total number of points {points.shape[0]}")

## Get boundary triangles

In [ ]:
is_boundary_triangle = np.zeros(triangles.shape[0])
for pidx in np.where(vertex_marker_is_boundary)[0]:
    if not is_extreme_boundary(edges, points, edge_marker_is_boundary, pidx)[0]:
        continue
    mask = np.any(triangles == pidx, axis=-1)
    is_boundary_triangle[mask] += 1

is_boundary_triangle = (is_boundary_triangle >= 2)

### Visualize the triangulation

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7, 7))
ax.axis("equal")


for tidx, element in enumerate(triangles):
    if is_boundary_triangle[tidx] == True:

        while vertex_marker_is_boundary[element][0] != 0:
            tmp = element[0]
            element[:-1] = element[1:]
            element[-1] = tmp

        tt = np.linspace(0, 1, 100)
        pts = points[element]
                
        a = directons[element[1]]
        b = directons[element[2]]
        
        v_22_1 = a[0]
        v_22_2 = a[1]
        
        v_32_1 = -b[0]
        v_32_2 = -b[1]
        

        e1_x, e1_y = mapping_function(tt, 1-tt, pts[0, 0], pts[0, 1], pts[1, 0], pts[1, 1], pts[2, 0], pts[2, 1], v_22_1, v_22_2, v_32_1, v_32_2)[:, 0]
        e2_x, e2_y = mapping_function(tt,    0, pts[0, 0], pts[0, 1], pts[1, 0], pts[1, 1], pts[2, 0], pts[2, 1], v_22_1, v_22_2, v_32_1, v_32_2)[:, 0]
        e3_x, e3_y = mapping_function(0,    tt, pts[0, 0], pts[0, 1], pts[1, 0], pts[1, 1], pts[2, 0], pts[2, 1], v_22_1, v_22_2, v_32_1, v_32_2)[:, 0]

        ax.plot(e1_x, e1_y, color="green", ls="-.")
        ax.plot(e2_x, e2_y, color="black")
        ax.plot(e3_x, e3_y, color="blue")
        
        # tmp = points[_]

        # a1 = tmp[0] - tmp[1]
        # a2 = tmp[0] - tmp[2]

        # print(np.cross(a1, a2))
                
    else:
        ax.plot(*points[element[[0, 1]]].T)
        ax.plot(*points[element[[1, 2]]].T)
        ax.plot(*points[element[[0, 2]]].T)
    
plt.show()

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(1)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points.T)

In [ ]:
matrix = np.zeros((points.shape[0], points.shape[0]))
b = np.zeros(points.shape[0])

for idx, element in enumerate(tqdm(triangles)):
    x1, x2, x3 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y1, y2, y3 = points[element[0], 1], points[element[1], 1], points[element[2], 1]

    if is_boundary_triangle[idx]:
        while vertex_marker_is_boundary[element][0] != 0:
            tmp = element[0]
            element[:-1] = element[1:]
            element[-1] = tmp
            
        pts = points[element]
        
        aa = directons[element[1]]
        bb = directons[element[2]]        
        
        v_22_1 = aa[0]
        v_22_2 = aa[1]
        
        v_32_1 = -bb[0]
        v_32_2 = -bb[1]

        FF = lagrange_linear_poisson_matrix_curvilinear(x1, y1, x2, y2, x3, y3, v_22_1, v_22_2, v_32_1, v_32_2)
        ff = lagrange_linear_poisson_right_values_curvilinear(x1, y1, x2, y2, x3, y3, v_22_1, v_22_2, v_32_1, v_32_2, *right_vals[element])
        

        for local_point in range(3):
            if vertex_marker_is_boundary[element[local_point]] == True:
                matrix[element[local_point], element[local_point]] = 1
                b[element[local_point]] *= 0
            else:
                for i in range(3):
                    matrix[element[local_point], element[i]] += FF[local_point, i]

                b[element[local_point]] -= ff[local_point]
                
    else:
            
        delta = x1 * y2 - x1 * y3 - x2 * y1 + x2 * y3 + x3 * y1 - x3 * y2

        P_1_x = (-y1 + y3) / delta
        P_1_y = (+x1 - x3) / delta

        P_2_x = (+y1 - y2) / delta
        P_2_y = (-x1 + x2) / delta

        arg = [P_1_x, P_2_x, P_1_y, P_2_y, *right_vals[element]]

        FF = lagrange_linear_poisson_matrix(x1, y1, x2, y2, x3, y3, )
        ff = lagrange_linear_poisson_right_values(x1, y1, x2, y2, x3, y3, *right_vals[element])

        for local_point in range(3):
            if vertex_marker_is_boundary[element[local_point]] == True:
                matrix[element[local_point], element[local_point]] = 1
                b[element[local_point]] *= 0
            else:
                for i in range(3):
                    matrix[element[local_point], element[i]] += FF[local_point, i]

                b[element[local_point]] -=  ff[local_point]

In [ ]:
F = np.linalg.solve(matrix, b)

In [ ]:
param = np.linspace(0, 1, 10)
tx, ty = np.meshgrid(param, param, indexing="ij")
mask = ty <= 1 - tx
tx = tx[mask]
ty = ty[mask]

In [ ]:
XX, YY, ZZ = [], [], []

for idx, element in enumerate(triangles):
    x1, x2, x3 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y1, y2, y3 = points[element[0], 1], points[element[1], 1], points[element[2], 1]


    if is_boundary_triangle[idx]:
        while vertex_marker_is_boundary[element][0] != 0:
            tmp = element[0]
            element[:-1] = element[1:]
            element[-1] = tmp
            
        pts = points[element]
        
        
        aa = directons[element[1]] 
        bb = directons[element[2]]
        
        
        v_22_1 = aa[0]
        v_22_2 = aa[1]
        
        v_32_1 = -bb[0]
        v_32_2 = -bb[1]

        e1_x, e1_y = mapping_function(tx, ty, 
                                      pts[0, 0], pts[0, 1], pts[1, 0], 
                                      pts[1, 1], pts[2, 0], pts[2, 1], 
                                      v_22_1, v_22_2, 
                                      v_32_1, v_32_2)[:, 0]


        XX.append(e1_x)
        YY.append(e1_y)
        
        ZZ.append(interpolation_function(tx, ty, 
                                         pts[0, 0], pts[0, 1], pts[1, 0], 
                                         pts[1, 1], pts[2, 0], pts[2, 1], 
                                         v_22_1, v_22_2, 
                                         v_32_1, v_32_2,
                                         F[element[0]], F[element[1]], F[element[2]]))
        
    else:
        XX.append(x1 + tx * (x2 - x1) + ty * (x3 - x1))
        YY.append(y1 + tx * (y2 - y1) + ty * (y3 - y1))
        ZZ.append(F[element[0]]*(1-tx-ty) + F[element[1]]*tx + F[element[2]]*ty)

# XX = np.concatenate(XX)
# YY = np.concatenate(YY)
# ZZ = np.concatenate(ZZ)

In [ ]:
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=xx,
                                y=yy,
                                z=zz,                                          
                                )    
    for xx, yy, zz in zip(XX, YY, ZZ) 
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()
